In [1]:
!pip install pydot

In [2]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc

Using TensorFlow backend.


In [3]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig

## 1. Import Embeddings (X) and Labels (Y)

### 1.1 Embeddings

In [4]:
now = time.time()
infile = None
infile = open("embeddings_7",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)

48.56441617012024


In [5]:
gc.collect()

0

In [6]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    i = sys.getsizeof(obj)
    if i >100000:
        print(var, i)

embeddings 589928


In [7]:
"""
now = time.time()
embeddings = {}
session = boto3.session.Session(region_name='us-east-1')
s3client = session.client('s3')
for i in ['embeddings1','embeddings2','embeddings3']:
          #,'embeddings2','embeddings3','embeddings4','embeddings5']:
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
print(time.time()-now)
"""

"\nnow = time.time()\nembeddings = {}\nsession = boto3.session.Session(region_name='us-east-1')\ns3client = session.client('s3')\nfor i in ['embeddings1','embeddings2','embeddings3']:\n          #,'embeddings2','embeddings3','embeddings4','embeddings5']:\n    response = s3client.get_object(Bucket='w210-mimic', Key=i)\n    body_string = response['Body'].read()\n    embeddings.update(cPickle.loads(body_string))\nprint(time.time()-now)\n"

In [8]:
embeddings_np = np.array(list(embeddings.items()))

In [9]:
embeddings_np[:,1].shape

(13441,)

In [10]:
HADM_ID_LIST = embeddings_np[:,0].copy()
embeddings = embeddings_np[:,1].copy()

In [11]:
sys.getsizeof(embeddings)

107624

In [12]:
len(embeddings), embeddings[0].shape

(13441, (14, 256, 768))

#### 1.1.1 Zero pad and truncate

In [13]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings

In [14]:
embeddings_padded = None
embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]

In [15]:
del(embeddings)

In [16]:
#Reshape embeddings
embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
#embeddings_concat = np.array([np.concatenate(i) for i in embeddings])

In [17]:
del(embeddings_padded)

In [18]:
len(embeddings_concat), len(embeddings_concat[1]), len(embeddings_concat[0][0])

(13441, 5120, 768)

In [19]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    i = sys.getsizeof(obj)
    if i >10000:
        print(var, i)

embeddings_np 215168
HADM_ID_LIST 107624
embeddings_concat 105704325248


In [69]:
del(embeddings)
del(embeddings_np)
del(HADM_ID_LIST)
del(arr)
del(CHAPTER_PIVOT_DF)
del(HADM_ID_DF)
del(CHAPTER_PIVOT_NP)
del(embeddings_concat)

In [20]:
del(embeddings_np)
del(arr)
del(i)

NameError: name 'arr' is not defined

### 1.2 Labels - Top 100 Dx code and 18 category

In [ ]:
CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/balanced_chapter_label.csv', sep=',', header = 0)\
                    .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

In [ ]:
#CHAPTER_PIVOT_DF.head()

In [ ]:
HADM_ID_DF = None
HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])

In [ ]:
CHAPTER_PIVOT_DF = HADM_ID_DF.set_index('HADM_ID')\
            .join(CHAPTER_PIVOT_DF, how='left')

In [ ]:
HADM_ID_LIST[:5]

In [ ]:
CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF)

In [ ]:
CHAPTER_PIVOT_NP.shape

In [ ]:
del(CHAPTER_PIVOT_DF)
#del(tmp)
del(HADM_ID_LIST)
del(HADM_ID_DF)

## 2. Split test and training

In [ ]:
CHAPTER_PIVOT_NP.shape

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.2)

In [ ]:
y_train2.shape, y_test2.shape

In [ ]:
X_train2.shape, X_test2.shape

In [ ]:
#X_train2 = X_train2.reshape(10752, 20, 256, 768)
#X_test2 = X_test2.reshape(2689, 20, 256, 768)

In [ ]:
#del(embeddings_concat)
#del(embeddings)
del(CHAPTER_PIVOT_NP)

In [ ]:
gc.collect()

## 3. Define LSTM pipeline

### 3.1 LSTM 1

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [38]:
import tensorflow as tf
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
deep_inputs = Input(shape=(20*256,768))
#LSTM_Layer_1 = LSTM(256)(deep_inputs)
LSTM_Layer_1 = Flatten()
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

In [77]:
opt = keras.optimizers.adam(lr=0.001)
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])
model.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1])

In [ ]:
history1 = model.fit(X_train2, y_train2, epochs=3, verbose=1, validation_split=0.1)

In [ ]:
score = model.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

In [ ]:
model_json = model.to_json()

with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5")

### 3.2 LSTM 2

In [ ]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
model1 = Model(inputs=deep_inputs, outputs=dense_layer_1)
opt = keras.optimizers.Adam(lr=0.001)
model1.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

In [ ]:
history1 = model1.fit(X_train2, y_train2, epochs=5, batch_size=32, verbose=1, validation_split=0.1)


Train on 9676 samples, validate on 1076 samples
Epoch 1/5
9676/9676 [==============================] - 1101s 114ms/step - loss: 0.5323 - get_f1: 0.6178 - val_loss: 0.5470 - val_get_f1: 0.6121
Epoch 3/5
9676/9676 [==============================] - 1101s 114ms/step - loss: 0.5255 - get_f1: 0.6311 - val_loss: 0.5408 - val_get_f1: 0.6008
Epoch 4/5
9676/9676 [==============================] - 1105s 114ms/step - loss: 0.5173 - get_f1: 0.6422 - val_loss: 0.5380 - val_get_f1: 0.6240
Epoch 5/5
9676/9676 [==============================] - 1099s 114ms/step - loss: 0.5087 - get_f1: 0.6534 - val_loss: 0.5387 - val_get_f1: 0.5939


In [ ]:
score = model1.evaluate(X_test2, y_test2, verbose=1)

print("Test F1:", score[1])

2689/2689 [==============================] - 137s 51ms/step
Test F1: 0.6105346083641052


In [ ]:
model_json = model1.to_json()

with open("model_num1.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model1.save_weights("model_num1.h5")

## LSTM 3

In [64]:
deep_inputs = Input(shape=(5120,768))
LSTM_Layer_1 = LSTM(256, return_sequences = True)(deep_inputs)
LSTM_Layer_2 = LSTM(128)(LSTM_Layer_1)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_2)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_1)
opt = keras.optimizers.Adam(lr=0.001)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

In [ ]:
history2 = model2.fit(X_train2, y_train2, epochs=3, batch_size=32, verbose=1, validation_split=0.1)

In [41]:
score = model2.evaluate(X_test2, y_test2, verbose=1)

print("Test F1:", score[1])

2689/2689 [==============================] - 180s 67ms/step
Test F1: 0.6257776021957397


In [ ]:
model_json = model2.to_json()

with open("model_num2_large.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model2.save_weights("model_num2_large.h5")

In [40]:
X_train2 = X_train2.reshape(10752,20,256,768)

In [41]:
X_train2.shape

(10752, 20, 256, 768)

In [8]:
from keras.layers import Conv2D, AveragePooling2D

In [115]:
pool_size = 4
filter_size = 8
num_filters = 16
deep_inputs = Input(shape=(20*256,768))
Conv1 = Conv1D(num_filters, filter_size)(deep_inputs)
Pool1 = AveragePooling1D(pool_size=pool_size)(Conv1)
flat = Flatten()(Pool1)
drop1 = Dropout(0.3)(flat)
dense_layer_2 = Dense(128, activation='relu')(drop1)
drop2 = Dropout(0.3)(dense_layer_2)
dense_layer_3 = Dense(64, activation='relu')(drop2)
dense_layer_4 = Dense(16, activation='sigmoid')(dense_layer_3)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_4)
opt = keras.optimizers.Adam(lr=0.0001)
model2.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1])

In [116]:
history2 = model2.fit(X_train2, y_train2, epochs=5, batch_size=32, verbose=1, validation_split=0.1)

Train on 9676 samples, validate on 1076 samples
Epoch 1/5
9676/9676 [==============================] - 684s 71ms/step - loss: 0.3613 - get_f1: 0.6497 - val_loss: 0.3470 - val_get_f1: 0.6612
Epoch 2/5
9676/9676 [==============================] - 679s 70ms/step - loss: 0.3321 - get_f1: 0.6759 - val_loss: 0.3408 - val_get_f1: 0.6702
Epoch 3/5
9676/9676 [==============================] - 679s 70ms/step - loss: 0.3263 - get_f1: 0.6812 - val_loss: 0.3411 - val_get_f1: 0.6664
Epoch 4/5
9676/9676 [==============================] - 683s 71ms/step - loss: 0.3227 - get_f1: 0.6846 - val_loss: 0.3307 - val_get_f1: 0.6795
Epoch 5/5
9676/9676 [==============================] - 689s 71ms/step - loss: 0.3158 - get_f1: 0.6926 - val_loss: 0.3246 - val_get_f1: 0.6865


In [ ]:
model2.summary()

In [117]:
score = model2.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 85s 32ms/step
Test Score: 0.30935339189098865
Test F1: 0.700751006603241


In [118]:
model_json = model2.to_json()

with open("model_num_cnn.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model2.save_weights("model_num_cnn.h5")

In [48]:
#def generate_aug_text(X_train2):
#    X_train2[0].shape

(5120, 768)

In [39]:
pool_size = 4
filter_size = 8
num_filters = 16
deep_inputs = Input(shape=(20*256,768))
Conv1 = Conv1D(num_filters, filter_size)(deep_inputs)
Pool1 = AveragePooling1D(pool_size=pool_size)(Conv1)
flat = Flatten()(Pool1)
drop1 = Dropout(0.3)(flat)
dense_layer_2 = Dense(128, activation='relu')(drop1)
drop2 = Dropout(0.3)(dense_layer_2)
dense_layer_3 = Dense(64, activation='relu')(drop2)
dense_layer_4 = Dense(16, activation='sigmoid')(dense_layer_3)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_4)
opt = keras.optimizers.Adam(lr=0.0001)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

In [40]:
history2 = model2.fit(X_train2, y_train2, epochs=5, batch_size=32, verbose=1, validation_split=0.1)

Train on 9676 samples, validate on 1076 samples
Epoch 1/5
9676/9676 [==============================] - 723s 75ms/step - loss: 0.5720 - get_f1: 0.5993 - val_loss: 0.5194 - val_get_f1: 0.6530
Epoch 2/5
9676/9676 [==============================] - 705s 73ms/step - loss: 0.5031 - get_f1: 0.6720 - val_loss: 0.4895 - val_get_f1: 0.6910
Epoch 3/5
9676/9676 [==============================] - 689s 71ms/step - loss: 0.4695 - get_f1: 0.7068 - val_loss: 0.4768 - val_get_f1: 0.7096
Epoch 4/5
9676/9676 [==============================] - 668s 69ms/step - loss: 0.4368 - get_f1: 0.7350 - val_loss: 0.4735 - val_get_f1: 0.6981
Epoch 5/5
9676/9676 [==============================] - 668s 69ms/step - loss: 0.4058 - get_f1: 0.7606 - val_loss: 0.4759 - val_get_f1: 0.7136


In [41]:
score = model2.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 83s 31ms/step
Test Score: 0.47465525195695246
Test F1: 0.6981337070465088


In [42]:
model_json = model2.to_json()

with open("model_num_cnn1.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model2.save_weights("model_num_cnn1.h5")

In [87]:
X_test2.shape

(2689,)

In [82]:
np.array([[0 for i in range(768)] for j in range(20*256)]).shape

(5120, 768)

In [67]:
aug_num = 3
BS = 10
#steps_per_epoch
SPE = (len(y_train2)*aug_num)/((1+aug_num)*BS)

def data_generator(X_train2, y_train2, BS = 6, aug_num = 4):
    N = len(y_train2)
    X_yield = []
    y_yield = []
    pad = [[0 for i in range(768)] for j in range(20*256)]
    #pad = np.zeros(20*256*768).reshape(20*256, 768)
    rand_LIST = np.array([i for i in range(20*256)])
    
    while True:

        for i in range(N):
            x = X_train2[i]
            y = y_train2[i]
            non_zero_x = sum(x[:,0]!=0)

            x_list = [x]
            for aug in range(aug_num):
                
                rand_NEW = rand_LIST[:non_zero_x]
                random.shuffle(rand_NEW)
                rand_NEW = np.append(rand_NEW,[n for n in range(non_zero_x, 20*256)], axis=0).astype('int')
                #print(rand_NEW)
                x_list.append([x[n] for n in rand_NEW.tolist()])

            y_list = [y for n in range(len(x_list))]
            
            if (i+1)%(BS+1) == 0:
                yield (np.array(X_yield), np.array(Y_yield))
            elif i%(BS+1) == 0:
                X_yield = x_list
                Y_yield = y_list
            else:
                X_yield = np.append(X_yield, x_list, axis=0).astype('float16')
                Y_yield = np.append(Y_yield, y_list, axis=0).astype('int') 
            

In [ ]:
x, y = data_generator(X_train2[:6], y_train2[:6], BS = 6, aug_num = 4)

In [30]:
sum(embeddings_concat[0][:,0]!=0)

3584

In [68]:
model2 = None
del(model2)

In [69]:
pool_size = 4
filter_size = 8
num_filters = 16
deep_inputs = Input(shape=(20*256,768))
Conv1 = Conv1D(num_filters, filter_size)(deep_inputs)
Pool1 = AveragePooling1D(pool_size=pool_size)(Conv1)
flat = Flatten()(Pool1)
drop1 = Dropout(0.3)(flat)
dense_layer_2 = Dense(128, activation='relu')(drop1)
drop2 = Dropout(0.3)(dense_layer_2)
dense_layer_3 = Dense(64, activation='relu')(drop2)
dense_layer_4 = Dense(16, activation='sigmoid')(dense_layer_3)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_4)
opt = keras.optimizers.Adam(lr=0.0001)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

In [ ]:
history2 = model2.fit(data_generator(X_train2, y_train2, BS, aug_num), steps_per_epoch = SPE, epochs=5, verbose=1)

Epoch 1/5
807/806 [==============================] - 2116s 3s/step - loss: 0.5555 - get_f1: 0.5951
Epoch 2/5
419/806 [==============>...............] - ETA: 16:49 - loss: 0.5115 - get_f1: 0.6503

In [ ]:
score = model2.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

In [ ]:
model_json = model2.to_json()

with open("model_num_cnn2.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model2.save_weights("model_num_cnn2.h5")

In [73]:
score

[0.4670162661317646, 0.7203916907310486]

In [75]:
history2.history

{'loss': [0.5554845749297845,
  0.5059249111563152,
  0.4873282749856715,
  0.4745975701413042,
  0.46655771638528715],
 'get_f1': [0.59505767, 0.65894014, 0.6815842, 0.69281137, 0.7003555]}

In [99]:
pool_size = 2
filter_size = 128
num_filters = 16
deep_inputs = Input(shape=(20*256,768))
Conv1 = Conv1D(num_filters, filter_size, strides=20)(deep_inputs)
Pool1 = AveragePooling1D(pool_size=pool_size)(Conv1)
pool_size = 2
filter_size = 32
num_filters = 8
#Conv2 = Conv1D(num_filters, filter_size, strides=1)(Pool1)
#Pool2 = AveragePooling1D(pool_size=pool_size)(Conv2)
flat = Flatten()(Pool1)
drop1 = Dropout(0.3)(flat)
dense_layer_2 = Dense(64, activation='relu')(drop1)
drop2 = Dropout(0.3)(dense_layer_2)
dense_layer_3 = Dense(32, activation='relu')(drop2)
dense_layer_4 = Dense(16, activation='sigmoid')(dense_layer_3)
model3 = Model(inputs=deep_inputs, outputs=dense_layer_4)
opt = keras.optimizers.Adam(lr=0.001)
model3.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1])

In [100]:
model3.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        (None, 5120, 768)         0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 250, 16)           1572880   
_________________________________________________________________
average_pooling1d_34 (Averag (None, 125, 16)           0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 2000)              0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                128064    
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)                0  

In [ ]:
history3 = model3.fit(X_train2, y_train2, epochs=5, batch_size=32, verbose=1, validation_split=0.1)

In [ ]:
score = model3.evaluate(X_test2.reshape(2689*20,256,768), y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

In [100]:
# define the model
def define_model():
    # channel
    def channel(inputs):
        conv1 = Conv1D(filters=16, kernel_size=8, activation='relu')(inputs)
        drop1 = Dropout(0.5)(conv1)
        pool1 = MaxPooling1D(pool_size=4)(drop1)
        return Flatten()(pool1)
    #Input Channel
    input1 = Input(shape=(256,768))
    input2 = Input(shape=(256,768))
    input3 = Input(shape=(256,768))
    input4 = Input(shape=(256,768))
    input5 = Input(shape=(256,768))
    input6 = Input(shape=(256,768))
    input7 = Input(shape=(256,768))
    input8 = Input(shape=(256,768))
    input9 = Input(shape=(256,768))
    input10 = Input(shape=(256,768))
    input11 = Input(shape=(256,768))
    input12 = Input(shape=(256,768))
    input13 = Input(shape=(256,768))
    input14 = Input(shape=(256,768))
    input15 = Input(shape=(256,768))
    input16 = Input(shape=(256,768))
    input17 = Input(shape=(256,768))
    input18 = Input(shape=(256,768))
    input19 = Input(shape=(256,768))
    input20 = Input(shape=(256,768))
    
    #flat layer
    flat1 = channel(input1)
    flat2 = channel(input2)
    flat3 = channel(input3)
    flat4 = channel(input4)
    flat5 = channel(input5)
    flat6 = channel(input6)
    flat7 = channel(input7)
    flat8 = channel(input8)
    flat9 = channel(input9)
    flat10 = channel(input10)
    flat11 = channel(input11)
    flat12 = channel(input12)
    flat13 = channel(input13)
    flat14 = channel(input14)
    flat15 = channel(input15)
    flat16 = channel(input16)
    flat17 = channel(input17)
    flat18 = channel(input18)
    flat19 = channel(input19)
    flat20 = channel(input20)

    merged = concatenate([flat1, flat2, flat3, flat4, flat5,
                         flat6, flat7, flat8, flat9, flat10,
                         flat11, flat12, flat13, flat14, flat15,
                         flat16, flat17, flat18, flat19, flat20])
    # interpretation
    dense1 = Dense(128, activation='relu')(merged)
    outputs = Dense(16, activation='sigmoid')(dense1)
    
    model = Model(inputs=[input1, input2, input3, input4, input5,
                         input6, input7, input8, input9, input10,
                         input11, input12, input13, input14, input15,
                         input16, input17, input18, input19, input20], outputs=outputs)
    # compile
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt', metrics=[get_f1])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [101]:
model3 = define_model()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_196 (InputLayer)          (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_197 (InputLayer)          (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_198 (InputLayer)          (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_199 (InputLayer)          (None, 256, 768)     0                                            
___________________________________________________________________________________________

In [106]:
print(['X_train2[:,'+str(i)+'].shape' for i in range(20)])

['X_train2[:,0].shape', 'X_train2[:,1].shape', 'X_train2[:,2].shape', 'X_train2[:,3].shape', 'X_train2[:,4].shape', 'X_train2[:,5].shape', 'X_train2[:,6].shape', 'X_train2[:,7].shape', 'X_train2[:,8].shape', 'X_train2[:,9].shape', 'X_train2[:,10].shape', 'X_train2[:,11].shape', 'X_train2[:,12].shape', 'X_train2[:,13].shape', 'X_train2[:,14].shape', 'X_train2[:,15].shape', 'X_train2[:,16].shape', 'X_train2[:,17].shape', 'X_train2[:,18].shape', 'X_train2[:,19].shape']


In [108]:
history3 = model3.fit([X_train2[:,0], X_train2[:,1], X_train2[:,2], X_train2[:,3], X_train2[:,4], X_train2[:,5], X_train2[:,6], X_train2[:,7], 
                       X_train2[:,8], X_train2[:,9], X_train2[:,10], X_train2[:,11], X_train2[:,12], X_train2[:,13], X_train2[:,14], X_train2[:,15], 
                       X_train2[:,16], X_train2[:,17], X_train2[:,18], X_train2[:,19]], 
                      y_train2, epochs=3, batch_size=32, verbose=1, validation_split=0.1)

Train on 9676 samples, validate on 1076 samples
Epoch 1/3
9676/9676 [==============================] - 514s 53ms/step - loss: 0.5495 - get_f1: 0.6114 - val_loss: 0.5290 - val_get_f1: 0.6529
Epoch 2/3
9676/9676 [==============================] - 513s 53ms/step - loss: 0.5062 - get_f1: 0.6623 - val_loss: 0.5173 - val_get_f1: 0.6737
Epoch 3/3
9676/9676 [==============================] - 513s 53ms/step - loss: 0.4833 - get_f1: 0.6870 - val_loss: 0.5087 - val_get_f1: 0.6365


In [109]:
score = model3.evaluate([X_test2[:,0], X_test2[:,1], X_test2[:,2], X_test2[:,3], X_test2[:,4], X_test2[:,5], X_test2[:,6], X_test2[:,7], X_test2[:,8], 
                         X_test2[:,9], X_test2[:,10], X_test2[:,11], X_test2[:,12], X_test2[:,13], X_test2[:,14], X_test2[:,15], X_test2[:,16], 
                         X_test2[:,17], X_test2[:,18], X_test2[:,19]], y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 100s 37ms/step
Test Score: 0.508201044237095
Test F1: 0.6530631184577942


In [112]:
model_json = model3.to_json()

with open("model_multichannel1.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model3.save_weights("model_multichannel1.h5")

In [34]:
# define the model
def define_model2():
    # channel
    def channel(inputs):
        conv1 = Conv1D(filters=16, kernel_size=8, activation='relu')(inputs)
        drop1 = Dropout(0.5)(conv1)
        pool1 = MaxPooling1D(pool_size=4)(drop1)
        return Flatten()(pool1)
    #Input Channel
    input1 = Input(shape=(256,768))
    input2 = Input(shape=(256,768))
    input3 = Input(shape=(256,768))
    input4 = Input(shape=(256,768))
    input5 = Input(shape=(256,768))
    input6 = Input(shape=(256,768))
    input7 = Input(shape=(256,768))
    input8 = Input(shape=(256,768))
    input9 = Input(shape=(256,768))
    input10 = Input(shape=(256,768))
    input11 = Input(shape=(256,768))
    input12 = Input(shape=(256,768))
    input13 = Input(shape=(256,768))
    input14 = Input(shape=(256,768))
    input15 = Input(shape=(256,768))
    input16 = Input(shape=(256,768))
    input17 = Input(shape=(256,768))
    input18 = Input(shape=(256,768))
    input19 = Input(shape=(256,768))
    input20 = Input(shape=(256,768))
    
    #flat layer
    flat1 = channel(input1)
    flat2 = channel(input2)
    flat3 = channel(input3)
    flat4 = channel(input4)
    flat5 = channel(input5)
    flat6 = channel(input6)
    flat7 = channel(input7)
    flat8 = channel(input8)
    flat9 = channel(input9)
    flat10 = channel(input10)
    flat11 = channel(input11)
    flat12 = channel(input12)
    flat13 = channel(input13)
    flat14 = channel(input14)
    flat15 = channel(input15)
    flat16 = channel(input16)
    flat17 = channel(input17)
    flat18 = channel(input18)
    flat19 = channel(input19)
    flat20 = channel(input20)

    merged = concatenate([flat1, flat2, flat3, flat4, flat5,
                         flat6, flat7, flat8, flat9, flat10,
                         flat11, flat12, flat13, flat14, flat15,
                         flat16, flat17, flat18, flat19, flat20])
    # interpretation
    dense1 = Dense(512, activation='relu')(merged)
    dense1 = Dense(128, activation='relu')(dense1)
    outputs = Dense(16, activation='sigmoid')(dense1)
    
    model = Model(inputs=[input1, input2, input3, input4, input5,
                         input6, input7, input8, input9, input10,
                         input11, input12, input13, input14, input15,
                         input16, input17, input18, input19, input20], outputs=outputs)
    # compile
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [37]:
model4 = define_model2()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 256, 768)     0                                            
____________________________________________________________________________________________

In [117]:
history4 = model4.fit([X_train2[:,0], X_train2[:,1], X_train2[:,2], X_train2[:,3], X_train2[:,4], X_train2[:,5], X_train2[:,6], X_train2[:,7], 
                       X_train2[:,8], X_train2[:,9], X_train2[:,10], X_train2[:,11], X_train2[:,12], X_train2[:,13], X_train2[:,14], X_train2[:,15], 
                       X_train2[:,16], X_train2[:,17], X_train2[:,18], X_train2[:,19]], 
                      y_train2, epochs=4, batch_size=32, verbose=1, validation_split=0.15)

Train on 9139 samples, validate on 1613 samples
Epoch 1/4
9139/9139 [==============================] - 432s 47ms/step - loss: 0.5558 - get_f1: 0.6016 - val_loss: 0.5295 - val_get_f1: 0.6041
Epoch 2/4
9139/9139 [==============================] - 431s 47ms/step - loss: 0.5127 - get_f1: 0.6488 - val_loss: 0.5188 - val_get_f1: 0.6391
Epoch 3/4
9139/9139 [==============================] - 432s 47ms/step - loss: 0.5004 - get_f1: 0.6652 - val_loss: 0.5200 - val_get_f1: 0.6746
Epoch 4/4
9139/9139 [==============================] - 434s 47ms/step - loss: 0.4856 - get_f1: 0.6836 - val_loss: 0.5099 - val_get_f1: 0.6711


In [118]:
score = model4.evaluate([X_test2[:,0], X_test2[:,1], X_test2[:,2], X_test2[:,3], X_test2[:,4], X_test2[:,5], X_test2[:,6], X_test2[:,7], X_test2[:,8], 
                         X_test2[:,9], X_test2[:,10], X_test2[:,11], X_test2[:,12], X_test2[:,13], X_test2[:,14], X_test2[:,15], X_test2[:,16], 
                         X_test2[:,17], X_test2[:,18], X_test2[:,19]], y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 100s 37ms/step
Test Score: 0.5161558028190009
Test F1: 0.676522433757782


In [ ]:
history4 = model4.fit([X_train2[:,19-i] for i in range(20)], 
                      y_train2, epochs=4, batch_size=32, verbose=1, validation_split=0.15)

Train on 9139 samples, validate on 1613 samples
Epoch 1/4
9139/9139 [==============================] - 515s 56ms/step - loss: 0.5488 - get_f1: 0.6030 - val_loss: 0.5394 - val_get_f1: 0.6091
Epoch 2/4
9139/9139 [==============================] - 512s 56ms/step - loss: 0.5314 - get_f1: 0.6155 - val_loss: 0.5374 - val_get_f1: 0.6019
Epoch 3/4
1824/9139 [====>.........................] - ETA: 6:04 - loss: 0.5135 - get_f1: 0.6336

In [126]:
history4.history

{'val_loss': [0.5393717672909112,
  0.5373929816414241,
  0.5392910511280835,
  0.5341873778294719],
 'val_get_f1': [0.6091227531433105,
  0.6019048690795898,
  0.6019054651260376,
  0.6046615242958069],
 'loss': [0.5487791633714222,
  0.5313684105260039,
  0.521833579406421,
  0.5099424986756335],
 'get_f1': [0.6029823, 0.6155106, 0.6280121, 0.6425677]}

In [127]:
score = model4.evaluate([X_test2[:,0], X_test2[:,1], X_test2[:,2], X_test2[:,3], X_test2[:,4], X_test2[:,5], X_test2[:,6], X_test2[:,7], X_test2[:,8], 
                         X_test2[:,9], X_test2[:,10], X_test2[:,11], X_test2[:,12], X_test2[:,13], X_test2[:,14], X_test2[:,15], X_test2[:,16], 
                         X_test2[:,17], X_test2[:,18], X_test2[:,19]], y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 81s 30ms/step
Test Score: 0.5558409102125299
Test F1: 0.6344249248504639


In [38]:
# define the model
def define_model3():
    # channel
    def channel(inputs):
        conv1 = Conv1D(filters=16, kernel_size=8, activation='relu')(inputs)
        drop1 = Dropout(0.5)(conv1)
        pool1 = AveragePooling1D(pool_size=4)(drop1)
        return Flatten()(pool1)
    #Input Channel
    input1 = Input(shape=(256,768))
    input2 = Input(shape=(256,768))
    input3 = Input(shape=(256,768))
    input4 = Input(shape=(256,768))
    input5 = Input(shape=(256,768))
    input6 = Input(shape=(256,768))
    input7 = Input(shape=(256,768))
    input8 = Input(shape=(256,768))
    input9 = Input(shape=(256,768))
    input10 = Input(shape=(256,768))
    input11 = Input(shape=(256,768))
    input12 = Input(shape=(256,768))
    input13 = Input(shape=(256,768))
    input14 = Input(shape=(256,768))
    input15 = Input(shape=(256,768))
    input16 = Input(shape=(256,768))
    input17 = Input(shape=(256,768))
    input18 = Input(shape=(256,768))
    input19 = Input(shape=(256,768))
    input20 = Input(shape=(256,768))
    
    #flat layer
    flat1 = channel(input1)
    flat2 = channel(input2)
    flat3 = channel(input3)
    flat4 = channel(input4)
    flat5 = channel(input5)
    flat6 = channel(input6)
    flat7 = channel(input7)
    flat8 = channel(input8)
    flat9 = channel(input9)
    flat10 = channel(input10)
    flat11 = channel(input11)
    flat12 = channel(input12)
    flat13 = channel(input13)
    flat14 = channel(input14)
    flat15 = channel(input15)
    flat16 = channel(input16)
    flat17 = channel(input17)
    flat18 = channel(input18)
    flat19 = channel(input19)
    flat20 = channel(input20)

    merged = concatenate([flat1, flat2, flat3, flat4, flat5,
                         flat6, flat7, flat8, flat9, flat10,
                         flat11, flat12, flat13, flat14, flat15,
                         flat16, flat17, flat18, flat19, flat20])
    # interpretation
    dense1 = Dense(128, activation='relu')(merged)
    outputs = Dense(16, activation='sigmoid')(dense1)
    
    model = Model(inputs=[input1, input2, input3, input4, input5,
                         input6, input7, input8, input9, input10,
                         input11, input12, input13, input14, input15,
                         input16, input17, input18, input19, input20], outputs=outputs)
    # compile
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [43]:
model5 = define_model3()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 256, 768)     0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 256, 768)     0                                            
____________________________________________________________________________________________

In [44]:
history5 = model5.fit([X_train2[:,i] for i in range(20)], 
                      y_train2, epochs=8, batch_size=32, verbose=1, validation_split=0.1)

Train on 9676 samples, validate on 1076 samples
Epoch 1/8
9676/9676 [==============================] - 386s 40ms/step - loss: 0.5397 - get_f1: 0.6130 - val_loss: 0.5189 - val_get_f1: 0.6459
Epoch 2/8
9676/9676 [==============================] - 383s 40ms/step - loss: 0.5001 - get_f1: 0.6652 - val_loss: 0.5076 - val_get_f1: 0.6620
Epoch 3/8
9676/9676 [==============================] - 384s 40ms/step - loss: 0.4831 - get_f1: 0.6835 - val_loss: 0.4999 - val_get_f1: 0.6619
Epoch 4/8
9676/9676 [==============================] - 385s 40ms/step - loss: 0.4656 - get_f1: 0.7013 - val_loss: 0.5112 - val_get_f1: 0.6563
Epoch 5/8
9676/9676 [==============================] - 380s 39ms/step - loss: 0.4497 - get_f1: 0.7147 - val_loss: 0.5112 - val_get_f1: 0.6495
Epoch 6/8
9676/9676 [==============================] - 386s 40ms/step - loss: 0.4285 - get_f1: 0.7332 - val_loss: 0.5186 - val_get_f1: 0.6596
Epoch 7/8
9676/9676 [==============================] - 385s 40ms/step - loss: 0.4056 - get_f1: 0.749

In [45]:
score = model5.evaluate([X_test2[:,0], X_test2[:,1], X_test2[:,2], X_test2[:,3], X_test2[:,4], X_test2[:,5], X_test2[:,6], X_test2[:,7], X_test2[:,8], 
                         X_test2[:,9], X_test2[:,10], X_test2[:,11], X_test2[:,12], X_test2[:,13], X_test2[:,14], X_test2[:,15], X_test2[:,16], 
                         X_test2[:,17], X_test2[:,18], X_test2[:,19]], y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 79s 29ms/step
Test Score: 0.5448296156546669
Test F1: 0.6483700275421143


In [47]:
model_json = model5.to_json()

with open("model_multichannel2.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model5.save_weights("model_multichannel2.h5")

In [ ]:
model6 = None
help(Conv1D)

In [105]:
# define the model
def define_model4():
    # channel
    def channel(inputs):
        conv1 = Conv1D(filters=16, kernel_size=32, activation='relu')(inputs)
        drop1 = Dropout(0.5)(conv1)
        pool1 = AveragePooling1D(pool_size=4)(drop1)
        return Flatten()(pool1)
    #Input Channel
    input1 = Input(shape=(20*256,768))
    input2 = Input(shape=(20*256,768))
    input3 = Input(shape=(20*256,768))
    input4 = Input(shape=(20*256,768))
    
    #flat layer
    flat1 = channel(input1)
    flat2 = channel(input2)
    flat3 = channel(input3)
    flat4 = channel(input4)

    merged = concatenate([flat1, flat2, flat3, flat4])
    # interpretation
    dense1 = Dense(128, activation='relu')(merged)
    drop2 = Dropout(0.3)(dense1)
    outputs = Dense(16, activation='sigmoid')(dense1)
    
    model = Model(inputs=[input1, input2, input3, input4], outputs=outputs)
    # compile
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [106]:
model6 = define_model4()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           (None, 5120, 768)    0                                            
__________________________________________________________________________________________________
input_51 (InputLayer)           (None, 5120, 768)    0                                            
__________________________________________________________________________________________________
input_52 (InputLayer)           (None, 5120, 768)    0                                            
__________________________________________________________________________________________________
input_53 (InputLayer)           (None, 5120, 768)    0                                            
___________________________________________________________________________________________

In [107]:
history6 = model6.fit([X_train2 for i in range(4)], 
                      y_train2, epochs=2, batch_size=32, verbose=1, validation_split=0.1)

Train on 9676 samples, validate on 1076 samples
Epoch 1/2
9676/9676 [==============================] - 1803s 186ms/step - loss: 0.4289 - get_f1: 0.5994 - val_loss: 0.4329 - val_get_f1: 0.5949
Epoch 2/2
9676/9676 [==============================] - 1783s 184ms/step - loss: 0.4659 - get_f1: 0.5822 - val_loss: 0.4867 - val_get_f1: 0.5578


In [108]:
score = model6.evaluate([X_test2 for i in range(4)], y_test2, verbose=1)

print("Test Score:", score[0])
print("Test F1:", score[1])

2689/2689 [==============================] - 349s 130ms/step
Test Score: 0.4708817114658008
Test F1: 0.5768540501594543
